In [1]:
import numpy as np
import pandas as pd
from equiflow import TableZero

In [2]:
np.random.seed(42)
n = 100
data = {
    'age': np.random.randint(18, 80, size=n),
    'race': np.random.choice(['White', 'Black', 'Asian', 'Hispanic', None],
                             size=n),
    'sex': np.random.choice(['Male', 'Female'], size=n),
    'english': np.random.choice(['Fluent', 'Limited'], size=n),
}

for i in range(1, 11):
    data[f'var{i}'] = np.random.randn(n)

df = pd.DataFrame(data)

In [3]:
data_0 = df.copy()
data_1 = data_0.loc[data_0.sex == "Female"]
data_2 = data_1.loc[data_1.english == "Fluent"]
data_3 = data_2.loc[data_2.race != "Hispanic"]

t = TableZero(dfs = [data_0, data_1, data_2, data_3],
              cols = ['race','sex', 'english'],
              format = 'N (%)',
              missingness = False,
              )

t1 = t.view_cohorts()
t1

Cohort                                    
                           0           1           2           3
Variable Value                                                  
Overall                  100          52          18          14
race     Black     13 (16.2)    8 (20.5)    3 (20.0)    3 (27.3)
         Hispanic  22 (27.5)    8 (20.5)    4 (26.7)     0 (0.0)
         White     25 (31.2)   11 (28.2)    5 (33.3)    5 (45.5)
         Asian     20 (25.0)   12 (30.8)    3 (20.0)    3 (27.3)
sex      Female    52 (52.0)  52 (100.0)  18 (100.0)  14 (100.0)
         Male      48 (48.0)     0 (0.0)     0 (0.0)     0 (0.0)
english  Limited   58 (58.0)   34 (65.4)     0 (0.0)     0 (0.0)
         Fluent    42 (42.0)   18 (34.6)  18 (100.0)  14 (100.0)

In [4]:
t.view_flow()

Flow,0 to 1,1 to 2,2 to 3
,,,
"Inital, n",100,52,18
"Removed, n",48,34,4
"Result, n",52,18,14


In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(t1)

Cohort                                    
                           0           1           2           3
Variable Value                                                  
Overall                  100          52          18          14
race     Black     13 (16.2)    8 (20.5)    3 (20.0)    3 (27.3)
         Hispanic  22 (27.5)    8 (20.5)    4 (26.7)     0 (0.0)
         White     25 (31.2)   11 (28.2)    5 (33.3)    5 (45.5)
         Asian     20 (25.0)   12 (30.8)    3 (20.0)    3 (27.3)
sex      Female    52 (52.0)  52 (100.0)  18 (100.0)  14 (100.0)
         Male      48 (48.0)     0 (0.0)     0 (0.0)     0 (0.0)
english  Limited   58 (58.0)   34 (65.4)     0 (0.0)     0 (0.0)
         Fluent    42 (42.0)   18 (34.6)  18 (100.0)  14 (100.0)